**Name:** Alex Medina

**File:** Spectrum Generator with ALL Clouds

## **SECTION 0:** Setup

In [30]:
# Begin
import os
os.environ['picaso_refdata'] = r'C:\Users\Alex\Desktop\Picaso\picaso\reference' # THIS MUST GO BEFORE YOUR IMPORT STATEMENT
os.environ['PYSYN_CDBS'] = r'C:\Users\Alex\Desktop\Picaso\grp\redcat\trds' # This is for the stellar data discussed below.

#General
import numpy as np
import pandas as pd
import astropy.units as u
import matplotlib.pyplot as plt
from pathlib import Path
from itertools import product

# Picaso
from picaso import justdoit as jdi
from picaso import justplotit as jpi

# Virga
from virga import justdoit as vj
from virga import justplotit as cldplt

# Other
from bokeh.models import Legend
from bokeh.palettes import Category10
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

## **SECTION 1:** Helper methods

In [2]:
def compute_mmw(profile_dict, fallback=2.36):
    """
    Computing mmw from Sonora profile if present; else use solar mmw.
    """
    mu = profile_dict.get("mu") or profile_dict.get("MU")
    if mu is None:
        return float(fallback) # Solar mmw
    mu = np.asarray(mu, float)
    if mu.ndim == 0:
        return float(mu)
    return float(np.mean(mu))

## **SECTION 2:** Main function

In [3]:
def bd_spectrum(waverange, gravity, Teff, kzz, fsed, mh, R,
                cloud_list, opacity_db, sonora_db, virga_db):
    """
    Compute a BD emission spectrum with Virga clouds.
    """

    # Opacity & inputs
    opa = jdi.opannection(wave_range=list(waverange), filename_db=opacity_db)
    bd = jdi.inputs(calculation="browndwarf")

    # Basic inputs
    bd.phase_angle(0)
    bd.gravity(gravity, gravity_unit=u.Unit('m/s**2'))
    bd.sonora(sonora_db, Teff)

    # Inject Kzz (must match pressure grid length)
    prof = bd.inputs['atmosphere']['profile']
    P = np.asarray(prof["pressure"], float)
    T = np.asarray(prof["temperature"], float)
    mmw = compute_mmw(prof)
    
    bd.inputs["atmosphere"]["profile"]["kz"] = [float(kzz)] * len(P)

    # Clouds
    bd.virga(cloud_list, virga_db, fsed=float(fsed), mh=mh, mmw=mmw)

    out = bd.spectrum(opa, full_output=True)

    # Convert to F_nu, then regrid to constant R in wavenumber
    wn, th = out["wavenumber"], out["thermal"]  # cm^-1 and erg/cm^2/s/cm
    # Convert to F_nu on a wavelength grid first
    wm = 1e4 / wn
    flamy    = th * 1e-8  # per angstrom instead of per cm
    sp = jdi.psyn.ArraySpectrum(wm, flamy, 
                                waveunits='um',
                                fluxunits='FLAM')
    sp.convert('um')
    sp.convert('Fnu')  # erg/cm^2/s/Hz

    wav_um, F_nu = sp.wave, sp.flux # micron and erg/cm^2/s/Hz
    out['fluxnu'] = F_nu

    # Regrid at constant resolving power in WAVENUMBER space
    k_cm1_in = 1e4 / wav_um
    k_cm1_rg, F_nu_rg = jdi.mean_regrid(k_cm1_in, F_nu, R=R)  # returns k in cm^-1

    # Convert back to wavelength (micron) for plotting
    lam_um_rg = 1e4 / k_cm1_rg
    # Ensure ascending wavelength for plotting
    order = np.argsort(lam_um_rg)
    lam_um_rg = lam_um_rg[order]
    F_nu_rg   = F_nu_rg[order]

    out['regridx'] = lam_um_rg
    out['regridy'] = F_nu_rg
    return lam_um_rg, F_nu_rg

## **SECTION 3:** Configuration

In [4]:
# To see what clouds are availible
vj.available()

['Al2O3',
 'CH4',
 'CaAl12O19',
 'CaTiO3',
 'Cr',
 'Fe',
 'H2O',
 'KCl',
 'Mg2SiO4',
 'MgSiO3',
 'MnS',
 'NH3',
 'Na2S',
 'SiO2',
 'TiO2',
 'ZnS']

In [18]:
# Directories
input_dir  = r"C:\Users\Alex\Desktop\Picaso\data\sonora" # Sonora db
virga_dir  = r"C:\Users\Alex\Desktop\Picaso\data\virga" # Virga
opacit_dir = None # Opacity db
# WILL CHANGE ONCE RUNNING ON NEWTON AND TP CORRECT
# r"/groups/tkaralidi/opacity_500k_for_R5000_egpoutput.db"
#output_dir = r"C:\Users\Alex\Desktop\Picaso\outputs" # Output store
output_dir = Path(r"C:\Users\Alex\Desktop\Picaso\outputs")

# Wavelength and resolution
wav_range       = (0.3, 3.0) # microns
res_R           = 300 
# WILL CHANGE ONCE RUNNING ON NEWTON AND TP CORRECT
# res_R = 5000

# Cloud microphysics
MH              = 1.0   # [M/H] metallicity factor ~ solar
clouds          = ['Cr','MgSiO3','MnS']

# Grid space
Teff_range      = (700, 2000)     # K
gravity_range   = (50.0, 1500.0)  # m/s^2
logg_range      = (1.0, 10.0)
fsed_range      = (1.0, 4.0)      # dimensionless
Kzz_range       = (1e8, 3e10)     # cm^2/s

# Sampling
seed            = 43
N_spectra       = 5 # Number of spectra to generate for sampling

# Knobs
save_per_case   = True   # Also save per-spectrum files with friendly names
VERBOSE         = True

## **SECTION 4:** Execute ONE Model

In [19]:
# Since just ONE model, we need just ONE parameter set
Teff_one      = 1500     # K
gravity_one   = 100      # m/s^2
fsed_one      = 1.0      # dimensionless
kzz_one   = 1e10                   # cm^2/s

# Print parameters
if VERBOSE:
    print("Chosen single-model parameters:")
    print(f"Teff  = {Teff_one:.1f} K")
    print(f"g     = {gravity_one:.1f} m/s^2")
    print(f"f_sed = {fsed_one:.2f}")
    print(f"Kzz   = {kzz_one:.3e} cm^2/s")
    print(f"[M/H] = {MH:.2f}")
    print(f"R     = {res_R:d}")
    print(f"Waverange (micron): {wav_range[0]}–{wav_range[1]}")

# Run ONE model (bd_spectrum returns wavenumber [cm^-1] and Fnu regridded at constant R in wavenumber)
wm, Fnu = bd_spectrum(waverange=wav_range,
                      gravity=gravity_one,
                      Teff=Teff_one,
                      kzz=kzz_one,
                      fsed=fsed_one,
                      mh=MH,
                      R=res_R,
                      cloud_list=clouds,
                      opacity_db=opacit_dir,
                      sonora_db=input_dir,
                      virga_db=virga_dir)

# For plotting
one_model_meta = {"Teff_K": Teff_one, "g_mps2": gravity_one, "fsed": fsed_one, "Kzz_cm2s": kzz_one,
                  "MH": MH, "R": res_R, "wav_um_min": float(wm.min()), "wav_um_max": float(wm.max())}

Chosen single-model parameters:
Teff  = 1500.0 K
g     = 100.0 m/s^2
f_sed = 1.00
Kzz   = 1.000e+10 cm^2/s
[M/H] = 1.00
R     = 300
Waverange (micron): 0.3–3.0


In [20]:
# Single plot

src = ColumnDataSource(data=dict(lam_um=wm, Fnu=Fnu))

title_txt = (f"BD Spectrum (Thermal Emission): "
             f"Teff={one_model_meta['Teff_K']:.0f} K, "
             f"g={one_model_meta['g_mps2']:.0f} m s^-2, "
             f"f_sed={one_model_meta['fsed']:.2f}, "
             f"Kzz={one_model_meta['Kzz_cm2s']:.1e} cm^2 s^-2")

p = figure(title=title_txt,
           x_axis_label="Wavelength (micron)",
           y_axis_label="Fnu (erg cm^-2 s^-1 Hz^-1)",
           sizing_mode="stretch_width",
           height=420,
           tools="pan,wheel_zoom,box_zoom,reset,save")

# Choose a simple line color from Category10
color = Category10[3][0]
p.line('lam_um', 'Fnu', source=src, line_width=2, color=color)

show(p)

## **SECTION 5:** Execute MULTIPLE Models

In [26]:
# Parameter space
# CAN EDIT TO PROPER PARAMETER SPACE
# AN ARRAY OF SPACED TEFF, GRAV, FSED, AND KZZ
Teff_p   = [1500]
grav_p   = [100]
fsed_p   = [1.0, 2.0]
kzz_p    = [1e9, 1e10]

grid = list(product(Teff_p, grav_p, fsed_p, kzz_p))
N_spectra = len(grid)

# Parameters
# WILL ADD A SPACING PRINT TO SEE HOW MANY SAMPLES PER
if VERBOSE:
    print(f"Sampling N={N_spectra} spectra at:")
    print(f"Teff_min={Teff_p[0]:.0f} K, Teff_max={Teff_p[-1]:.0f} K")
    print(f"grav_min={grav_p[0]:.0f} m s^-2, grav_max={grav_p[-1]:.0f} m s^-2")
    print(f"fsed_min={fsed_p[0]:.0f}, fsed_max={fsed_p[-1]:.0f}")
    print(f"Kzz_min={kzz_p[0]:.1e}, cm^2 s^-1, {kzz_p[-1]:.1e} cm^2 s^-1")

def run_bd_point(Teff, g, fsed, kzz):
    out = bd_spectrum(waverange=wav_range,
                      gravity=float(g),
                      Teff=float(Teff),
                      kzz=float(kzz),
                      fsed=float(fsed),
                      mh=MH,
                      R=res_R,
                      cloud_list=clouds,
                      opacity_db=opacit_dir,
                      sonora_db=input_dir,
                      virga_db=virga_dir)
    if not (isinstance(out, tuple) and len(out) >= 2):
        raise RuntimeError("bd_spectrum must return at least (x, Fnu).")
    lam_um = np.asarray(out[0], float)
    Fnu    = np.asarray(out[1], float)
    idx    = np.argsort(lam_um)
    return lam_um[idx], Fnu[idx]

multi_specs = []
lam_ref = None
for (Teff_i, g_i, fsed_i, kzz_i) in grid:
    lam_i, F_i = run_bd_point(Teff_i, g_i, fsed_i, kzz_i)

    if lam_ref is None:
        lam_ref = lam_i.copy()
    else:
        # If needed, interpolate to the reference wavelength grid
        if (lam_i.shape != lam_ref.shape) or (not np.allclose(lam_i, lam_ref, rtol=0, atol=1e-12)):
            F_i = np.interp(lam_ref, lam_i, F_i)
            lam_i = lam_ref

    multi_specs.append(dict(
        lam_um=lam_i,
        Fnu=F_i,
        Teff=float(Teff_i),
        g=float(g_i),
        fsed=float(fsed_i),
        kzz=float(kzz_i)
    ))

multi_meta = {
    "R":        res_R,
    "MH":       float(MH),
    "N":        int(N_spectra),
    "lam_range": (float(lam_ref[0]), float(lam_ref[-1])),
    "Teff_grid": np.array(Teff_p, float),
    "g_grid":    np.array(grav_p, float),
    "fsed_grid": np.array(fsed_p, float),
    "kzz_grid":  np.array(kzz_p, float),
}

Sampling N=4 spectra at:
Teff_min=1500 K, Teff_max=1500 K
grav_min=100 m s^-2, grav_max=100 m s^-2
fsed_min=1, fsed_max=2
Kzz_min=1.0e+09, cm^2 s^-1, 1.0e+10 cm^2 s^-1


In [32]:
# In the future will have a large range of TEFF AND GRAV
# SO WILL BE ABLE TO CHANGE

uniq_T = sorted({float(s["Teff"]) for s in multi_specs})
uniq_g = sorted({float(s["g"])    for s in multi_specs})

def fmt_range(vals, unit="", fmt="{:.0f}"):
    if len(vals) == 1:
        return f"{fmt.format(vals[0])}{unit}"
    return f"{fmt.format(min(vals))} to {fmt.format(max(vals))}{unit} (n={len(vals)})"

Teff_txt = fmt_range(uniq_T, unit=" K", fmt="{:.0f}")
g_txt    = fmt_range(uniq_g, unit=" m s^-2", fmt="{:.0f}")

title_txt = (f"BD spectra (thermal emisson):"
             f"N={multi_meta['N']},"
             f"Teff={Teff_txt}"
             f"g={g_txt}")

# Palette that can scale beyond 10 lines (repeats after 20)
# WILL ONLY BE RELEVANT FOR THIS BUT NOT FOR FUTURE
def choose_palette(n):
    if n <= 3:   # Category10[3] is more distinct than slicing [10]
        base = Category10[3]
        return base[:n]
    if n <= 10:
        return Category10[10][:n]
    if n <= 20:
        return Category20[20][:n]
    # Repeat
    reps = int(np.ceil(n/20))
    return (Category20[20] * reps)[:n]

palette = choose_palette(len(multi_specs))

# Figure
p = figure(title=title_txt,
           x_axis_label="Wavelength (micron)",
           y_axis_label="Fν (erg cm^-2 s^-1 Hz^-1)",
           sizing_mode="stretch_width",
           height=520,
           tools="pan,wheel_zoom,box_zoom,reset,save")

# If you later include multiple Teff/g, include them in the legend only when varied
vary_T = len(uniq_T) > 1
vary_g = len(uniq_g) > 1

for i, s in enumerate(multi_specs):
    src = ColumnDataSource(dict(lam_um=s["lam_um"], Fnu=s["Fnu"]))
    parts = []
    if vary_T: parts.append(f"T={s['Teff']:.0f} K")
    if vary_g: parts.append(f"g={s['g']:.0f} m s⁻²")
    parts.append(f"f_sed={s['fsed']:.2g}")
    parts.append(f"Kzz={s['kzz']:.1e}")
    label = ", ".join(parts)

    p.line('lam_um', 'Fnu', source=src, line_width=2,
           color=palette[i % len(palette)], legend_label=label)

p.legend.location = "top_right"
p.legend.click_policy = "hide"
p.legend.label_text_font_size = "10pt"

show(p)

## **SECTION 6:** Generating and Saving Multple Models in .NPZ

In [33]:
# Build arrays X (params) and Y (spectra), and confirm a common wavelength grid
N = len(multi_specs)
lam_ref = multi_specs[0]["lam_um"].astype(float)
L = lam_ref.size

# [Teff, g, f_sed, Kzz]
X = np.zeros((N, 4), dtype=float)
Y = np.zeros((N, L), dtype=float)

for i, s in enumerate(multi_specs):
    lam_i = s["lam_um"].astype(float)
    Fnu_i = s["Fnu"].astype(float)

    # If any spectrum differs in wavelength grid, interpolate to lam_ref
    if lam_i.shape != lam_ref.shape or not np.allclose(lam_i, lam_ref, rtol=0, atol=0):
        Fnu_i = np.interp(lam_ref, lam_i, Fnu_i)

    X[i, :] = [s["Teff"], s["g"], s["fsed"], s["kzz"]]
    Y[i, :] = Fnu_i

# Build a readable master filename (supports single or multiple Teff/g)
uniq_T = sorted({float(s["Teff"]) for s in multi_specs})
uniq_g = sorted({float(s["g"])    for s in multi_specs})

def _fmt_range(vals, key):
    if len(vals) == 1:
        return f"{int(vals[0])}"
    return f"{int(min(vals))}-{int(max(vals))}"

T_tag = _fmt_range(uniq_T, "T")
G_tag = _fmt_range(uniq_g, "G")

# Master file name (includes clouds tag)
master_name = f"T{T_tag}_G{G_tag}_N{N}_allclouds.npz"
master_path = output_dir / master_name

# Save master with a few helpful extras
np.savez_compressed(
    master_path,
    x=X,                       # (N,4): [Teff, g, f_sed, Kzz]
    y=Y,                       # (N,L): spectra on common lam_ref
    wavelength_um=lam_ref,     # (L,)
    clouds=np.array(clouds),   # list of cloud species used
    MH=float(MH),
    R=int(res_R),
)
print(f"Saved master: {master_path}")

# Save per-case files (short, readable names + allclouds tag)
if save_per_case:
    for i in range(N):
        Teff_i, g_i, fsed_i, kzz_i = X[i]
        fname = (
            f"T{int(Teff_i)}_G{int(g_i)}_fsed{fsed_i:.2f}_"
            f"Kzz{float(kzz_i):.2e}_allclouds.npz"
        )
        fpath = output_dir / fname
        np.savez_compressed(
            fpath,
            x=X[i],                   # (4,)
            y=Y[i],                   # (L,)
            wavelength_um=lam_ref,    # (L,)
            clouds=np.array(clouds),
            MH=float(MH),
            R=int(res_R),
        )
    print(f"Saved {N} per-case files to {output_dir}")

Saved master: C:\Users\Alex\Desktop\Picaso\outputs\T1500_G100_N4_allclouds.npz
Saved 4 per-case files to C:\Users\Alex\Desktop\Picaso\outputs


## **SECTION 7:** Sample Stats

In [38]:
def inspect_npz(npz_path, show_table=True, max_rows=10):
    """
    Print shapes, input ranges, and flux stats for a saved .npz.
    Works for both per-case (x=(4,), y=(L,)) and master (x=(N,4), y=(N,L)).
    """
    npz_path = Path(npz_path)
    d = np.load(npz_path)

    # Normalize to batch form
    lam = d["wavelength_um"].astype(float)
    x_raw = d["x"]
    y_raw = d["y"]

    if x_raw.ndim == 1:   # per-case
        X = x_raw[None, :].astype(float)   # (1,4)
        Y = y_raw[None, :].astype(float)   # (1,L)
    else:                 # master
        X = x_raw.astype(float)            # (N,4)
        Y = y_raw.astype(float)            # (N,L)

    # Shapes
    print(f"X shape: {X.shape}  (columns: Teff[K], g[m/s^2], f_sed[-], Kzz[cm^2/s])")
    print(f"Y shape: {Y.shape}  (flux_nu per spectrum)")
    print(f"λ shape: {lam.shape} (micron)\n")

    # Input ranges
    Teff = X[:, 0]; g = X[:, 1]; fsed = X[:, 2]; Kzz = X[:, 3]
    print("Input ranges:")
    print(f"  Teff [K]    : {np.min(Teff):.1f} to {np.max(Teff):.1f}")
    print(f"  g [m/s^2]   : {np.min(g):.2f} to {np.max(g):.2f}")
    print(f"  f_sed [-]   : {np.min(fsed):.2f} to {np.max(fsed):.2f}")
    print(f"  Kzz [cm^2/s]: {np.min(Kzz):.3e} to {np.max(Kzz):.3e}\n")

    # Flux stats across all spectra
    ymin = np.min(Y)
    ymax = np.max(Y)
    print("Flux stats across all spectra:")
    print(f"  min: {ymin:.6e}  max: {ymax:.6e}\n")

    # Optional small table of inputs
    if show_table:
        df = pd.DataFrame(
            X,
            columns=["Teff_K", "g_mps2", "f_sed", "Kzz_cm2s"],
            index=pd.Index(range(X.shape[0]), name="spectrum_idx"),
        )
        # Show a compact view
        print(df.head(max_rows).to_string())

    return X, Y, lam

In [40]:
# Examples
# Per-case file:
inspect_npz(r"C:\Users\Alex\Desktop\Picaso\outputs\T1500_G100_fsed1.00_Kzz1.00e+09_allclouds.npz")

# Master file:
#inspect_npz(r"C:\Users\Alex\Desktop\Picaso\outputs\T1700_G100_MH1.00_R300_N5.npz", show_table=True, max_rows=10)

X shape: (1, 4)  (columns: Teff[K], g[m/s^2], f_sed[-], Kzz[cm^2/s])
Y shape: (1, 691)  (flux_nu per spectrum)
λ shape: (691,) (micron)

Input ranges:
  Teff [K]    : 1500.0 to 1500.0
  g [m/s^2]   : 100.00 to 100.00
  f_sed [-]   : 1.00 to 1.00
  Kzz [cm^2/s]: 1.000e+09 to 1.000e+09

Flux stats across all spectra:
  min: 1.267876e-18  max: 5.060036e-07

              Teff_K  g_mps2  f_sed      Kzz_cm2s
spectrum_idx                                     
0             1500.0   100.0    1.0  1.000000e+09


(array([[1.5e+03, 1.0e+02, 1.0e+00, 1.0e+09]]),
 array([[2.55257089e-17, 3.22720641e-17, 3.45172169e-17, 1.44582451e-17,
         4.42199144e-17, 5.52013725e-17, 6.18737738e-17, 6.70762497e-17,
         6.69324405e-17, 3.28172845e-17, 2.56263369e-17, 8.23278912e-17,
         9.87830631e-17, 1.05767060e-16, 1.09101402e-16, 1.09407769e-16,
         1.06442384e-16, 9.97262643e-17, 8.84988991e-17, 7.01499088e-17,
         3.25152028e-17, 2.50371755e-17, 4.05700567e-17, 3.57189130e-17,
         3.18056360e-17, 2.96178352e-17, 2.62778109e-17, 1.53223275e-17,
         1.26787605e-18, 3.55110649e-18, 2.67431848e-17, 4.87389495e-17,
         6.75362805e-17, 9.31749329e-17, 1.38024991e-16, 1.89293699e-16,
         2.33231168e-16, 2.69116135e-16, 2.87062492e-16, 2.72623252e-16,
         1.94376128e-16, 7.58856153e-17, 1.49722592e-16, 2.40421467e-16,
         2.75591625e-16, 2.81994742e-16, 2.79269705e-16, 2.74081128e-16,
         2.69035845e-16, 2.65288251e-16, 2.63358676e-16, 2.63464657e-16,
   

## **SECTION 8:** Plot

In [35]:
def plot_npz_file(npz_path, idx=None, title=None):
    """
    Plot a single .npz file.
    - Per-case file: ignores idx (since it stores one spectrum).
    - Master file: set idx to choose which row to plot.
    """
    d = np.load(npz_path, allow_pickle=True)
    lam = d["wavelength_um"].astype(float)

    # Master vs per-case
    if d["y"].ndim == 2:
        if idx is None:
            idx = 0
        F = d["y"][idx].astype(float)
        x_row = d["x"][idx].astype(float)
    else:
        F = d["y"].astype(float)
        x_row = d["x"].astype(float)

    if title is None:
        Teff, g, fsed, kzz = x_row
        title = f"{Path(npz_path).name} — Teff={Teff:.0f}, g={g:.0f}, f_sed={fsed:.2f}, Kzz={kzz:.2e}"

    p = figure(title=title,
               x_axis_label="Wavelength (micron)",
               y_axis_label="Fnu (erg cm^-2 s^-1 Hz^-1)",
               sizing_mode="stretch_width", height=420,
               tools="pan,wheel_zoom,box_zoom,reset,save")
    src = ColumnDataSource(dict(lam_um=lam, Fnu=F))
    p.line('lam_um', 'Fnu', source=src, line_width=2)
    show(p)

def overlay_npz_from_folder(folder, max_plots=5):
    """
    Overlay up to max_plots spectra from per-case files OR randomly sample from master files.
    """
    folder = Path(folder)
    paths = sorted(folder.glob("*.npz"))
    if not paths:
        print("No .npz files found.")
        return

    p = figure(title=f"Overlay check — up to {max_plots} spectra",
               x_axis_label="Wavelength (micron)",
               y_axis_label="Fnu (erg cm^-2 s^-1 Hz^-1)",
               sizing_mode="stretch_width", height=520,
               tools="pan,wheel_zoom,box_zoom,reset,save")

    pal = Category10[max(3, min(10, max_plots))]
    plotted = 0

    for path in paths:
        if plotted >= max_plots:
            break
        d = np.load(path, allow_pickle=True)
        lam = d["wavelength_um"].astype(float)
        y = d["y"]

        # Per-case
        if y.ndim == 1:
            F = y.astype(float)
            p.line(lam, F, line_width=2, color=pal[plotted % len(pal)],
                   legend_label=path.name)
            plotted += 1

        # Master: sample a few rows
        else:
            rows = min(y.shape[0], max_plots - plotted)
            for i in range(rows):
                F = y[i].astype(float)
                p.line(lam, F, line_width=2, color=pal[plotted % len(pal)],
                       legend_label=f"{path.name} [i={i}]")
                plotted += 1
                if plotted >= max_plots:
                    break

    p.legend.location = "top_right"
    p.legend.click_policy = "hide"
    show(p)

In [37]:
# For a single file
plot_npz_file(r"C:\Users\Alex\Desktop\Picaso\outputs\T1500_G100_fsed1.00_Kzz1.00e+09_allclouds.npz")

# For an area of the master path
plot_npz_file(rf"{master_path}", idx=1)

# Overall
overlay_npz_from_folder(r"C:\Users\Alex\Desktop\Picaso\outputs", max_plots=6)